In [ ]:
class Subject:
    """Class representing a subject with name and grades."""

    def __init__(self, name):
        """Initialize a Subject with a name and empty grades list."""
        self.name = name
        self.grades = []

    def add_grade(self, grade):
        """Add a grade to the subject."""
        # Validate grade is between 0 and 100
        if not (0 <= grade <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self.grades.append(grade)

    def get_average(self):
        """Calculate average grade for the subject."""
        if not self.grades:
            return 0
        return sum(self.grades) / len(self.grades)

    def __str__(self):
        """String representation of the subject."""
        grades_str = ", ".join(str(grade) for grade in self.grades)
        average = self.get_average()
        return f"{self.name}: Grades [{grades_str}], Average: {average:.2f}"


class Student:
    """Class representing a student with ID, name and subjects."""

    # Class variable to keep track of the next available ID
    next_id = 1

    def __init__(self, name, student_id=None):
        """Initialize a Student with a name, ID, and empty subjects dictionary."""
        self.name = name
        # Assign provided ID or auto-generate one
        if student_id is not None:
            self.id = student_id
            # Update next_id if needed to avoid future collisions
            Student.next_id = max(Student.next_id, int(student_id) + 1)
        else:
            self.id = Student.next_id
            Student.next_id += 1
        self.subjects = {}  # Dictionary to store subject objects: {subject_name: Subject object}

    def add_subject(self, subject_name):
        """Add a new subject to the student."""
        if subject_name not in self.subjects:
            self.subjects[subject_name] = Subject(subject_name)
        return self.subjects[subject_name]

    def add_grade(self, subject_name, grade):
        """Add a grade to a specific subject."""
        # If subject doesn't exist, create it
        if subject_name not in self.subjects:
            self.add_subject(subject_name)

        # Add grade to the subject
        self.subjects[subject_name].add_grade(grade)

    def get_average(self):
        """Calculate overall average grade for all subjects."""
        if not self.subjects:
            return 0

        total = sum(subject.get_average() for subject in self.subjects.values())
        return total / len(self.subjects)

    def __str__(self):
        """String representation of the student."""
        result = f"Student ID: {self.id} | Name: {self.name}\n"
        for subject in self.subjects.values():
            result += f"  {subject}\n"
        result += f"Overall Average: {self.get_average():.2f}"
        return result


class GradeTracker:
    """Class to manage the student grade tracking system."""

    def __init__(self):
        """Initialize the GradeTracker with empty dictionaries."""
        self.students = {}  # Dictionary to store student objects by name: {student_name: Student object}
        self.students_by_id = {}  # Dictionary to store student objects by ID: {student_id: Student object}

    def add_student(self, name, student_id=None):
        """Add a new student to the tracker."""
        if name in self.students:
            print(f"Student '{name}' already exists.")
            return self.students[name]

        # Check if ID is provided and already exists
        if student_id is not None and student_id in self.students_by_id:
            print(f"Student ID {student_id} is already assigned to another student.")
            return None

        student = Student(name, student_id)
        self.students[name] = student
        self.students_by_id[student.id] = student
        print(f"Student '{name}' added successfully with ID: {student.id}.")
        return student

    def add_grade(self, student_identifier, subject_name, grade):
        """Add a grade for a specific subject for a student by name or ID."""
        try:
            grade = float(grade)  # Convert grade to float

            # Find student by ID or name
            student = None

            # Check if identifier is an ID
            if isinstance(student_identifier, str) and student_identifier.isdigit():
                student_id = int(student_identifier)
                if student_id in self.students_by_id:
                    student = self.students_by_id[student_id]

            # If not found by ID, check by name
            if student is None and student_identifier in self.students:
                student = self.students[student_identifier]

            # If student doesn't exist, add them (by name)
            if student is None:
                if isinstance(student_identifier, str) and not student_identifier.isdigit():
                    student = self.add_student(student_identifier)
                else:
                    raise ValueError(f"Student with identifier '{student_identifier}' not found.")

            # Add grade to student's subject
            student.add_grade(subject_name, grade)
            print(f"Grade {grade} added to {subject_name} for student ID {student.id} ({student.name}).")

        except ValueError as e:
            print(f"Error: {e}")

    def view_student(self, identifier):
        """View details of a specific student by name or ID."""
        # Check if identifier is an integer (ID)
        try:
            if isinstance(identifier, str) and identifier.isdigit():
                student_id = int(identifier)
                if student_id in self.students_by_id:
                    print(self.students_by_id[student_id])
                    return
        except:
            pass

        # If not found by ID or not an ID, try as name
        if identifier in self.students:
            print(self.students[identifier])
        else:
            print(f"Student with name or ID '{identifier}' not found.")

    def view_all_students(self):
        """View details of all students."""
        if not self.students:
            print("No students found.")
            return

        print("\n===== All Students =====")
        for student in self.students.values():
            print(f"\n{student}")
            print("=" * 30)

    def save_to_file(self, filename="student_data.txt"):
        """Save all student data to a file."""
        try:
            with open(filename, 'w') as file:
                for student_name, student in self.students.items():
                    # Write student ID and name
                    file.write(f"STUDENT:{student.id}:{student_name}\n")

                    # Write each subject and its grades
                    for subject_name, subject in student.subjects.items():
                        grades_str = ",".join(str(grade) for grade in subject.grades)
                        file.write(f"SUBJECT:{subject_name}:{grades_str}\n")

                    # Add a separator between students
                    file.write("END_STUDENT\n")

            print(f"Data saved successfully to {filename}.")
            return True

        except Exception as e:
            print(f"Error saving data: {e}")
            return False

    def load_from_file(self, filename="student_data.txt"):
        """Load student data from a file."""
        try:
            self.students = {}  # Clear existing data
            self.students_by_id = {}  # Clear existing ID mapping

            with open(filename, 'r') as file:
                current_student = None

                for line in file:
                    line = line.strip()

                    if line.startswith("STUDENT:"):
                        # Extract student ID and name, and create student object
                        parts = line[8:].split(":")  # Remove "STUDENT:" prefix
                        if len(parts) >= 2:
                            student_id = int(parts[0])
                            student_name = parts[1]
                            current_student = self.add_student(student_name, student_id)
                        else:
                            # Backward compatibility with old format
                            student_name = parts[0]
                            current_student = self.add_student(student_name)

                    elif line.startswith("SUBJECT:"):
                        # Extract subject name and grades
                        parts = line[8:].split(":")  # Remove "SUBJECT:" prefix
                        if len(parts) >= 2:
                            subject_name = parts[0]

                            # Add grades if they exist
                            if parts[1]:
                                grades = parts[1].split(",")
                                for grade_str in grades:
                                    if grade_str:  # Check if grade string is not empty
                                        try:
                                            grade = float(grade_str)
                                            current_student.add_grade(subject_name, grade)
                                        except ValueError:
                                            print(f"Warning: Invalid grade '{grade_str}' ignored.")

            print("Data loaded successfully.")
            return True

        except FileNotFoundError:
            print(f"File {filename} not found. Starting with an empty tracker.")
            return False

        except Exception as e:
            print(f"Error loading data: {e}")
            return False


def main():
    """Main function to run the Student Grade Tracker program."""
    tracker = GradeTracker()

    # Try to load existing data
    tracker.load_from_file()

    while True:
        print("\n===== Student Grade Tracker =====")
        print("1. Add Student")
        print("2. Add Subject and Grade")
        print("3. View Student")
        print("4. View All Students")
        print("5. Save and Exit")
        print("6. Exit without Saving")

        try:
            choice = input("\nEnter your choice (1-6): ")

            if choice == '1':
                name = input("Enter student name: ")
                custom_id = input("Enter student ID (leave blank for auto-generation): ")
                if custom_id.strip():
                    tracker.add_student(name, int(custom_id))
                else:
                    tracker.add_student(name)

            elif choice == '2':
                student_identifier = input("Enter student name or ID: ")
                subject_name = input("Enter subject name: ")
                grade = input("Enter grade (0-100): ")
                tracker.add_grade(student_identifier, subject_name, grade)

            elif choice == '3':
                identifier = input("Enter student name or ID: ")
                tracker.view_student(identifier)

            elif choice == '4':
                tracker.view_all_students()

            elif choice == '5':
                tracker.save_to_file()
                print("Goodbye!")
                break

            elif choice == '6':
                confirm = input("Are you sure you want to exit without saving? (y/n): ")
                if confirm.lower() == 'y':
                    print("Goodbye!")
                    break

            else:
                print("Invalid choice. Please enter a number between 1 and 6.")

        except Exception as e:
            print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

File student_data.txt not found. Starting with an empty tracker.

===== Student Grade Tracker =====
1. Add Student
2. Add Subject and Grade
3. View Student
4. View All Students
5. Save and Exit
6. Exit without Saving

Enter your choice (1-6): 1
Enter student name: taha
Enter student ID (leave blank for auto-generation): 123
Student 'taha' added successfully with ID: 123.

===== Student Grade Tracker =====
1. Add Student
2. Add Subject and Grade
3. View Student
4. View All Students
5. Save and Exit
6. Exit without Saving

Enter your choice (1-6): 1
Enter student name: moahmed
Enter student ID (leave blank for auto-generation): 
Student 'moahmed' added successfully with ID: 124.

===== Student Grade Tracker =====
1. Add Student
2. Add Subject and Grade
3. View Student
4. View All Students
5. Save and Exit
6. Exit without Saving

Enter your choice (1-6): 2
Enter student name or ID: 123
Enter subject name: math
Enter grade (0-100): 80
Grade 80.0 added to math for student ID 123 (taha).

==